In [ ]:
!pip install --quiet --upgrade tensorflow tensorflow-federated

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.9 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
import tensorflow_federated as tff
import pandas as pd
import numpy as np

# ***LOADING DATA***

In [ ]:
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
column_names = [
    "age", "workclass", "fnlwgt", "education", "education_num", "marital_status",
    "occupation", "relationship", "race", "sex", "capital_gain", "capital_loss",
    "hours_per_week", "native_country", "income"
]

data = pd.read_csv(url, header=None, names=column_names, na_values=" ?", skipinitialspace=True)
data.dropna(inplace=True)
data.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


# ***Horizontal Federated Learning***

In [ ]:
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
column_names = [
    "age", "workclass", "fnlwgt", "education", "education_num", "marital_status",
    "occupation", "relationship", "race", "sex", "capital_gain", "capital_loss",
    "hours_per_week", "native_country", "income"
]
data = pd.read_csv(url, header=None, names=column_names, na_values=" ?", skipinitialspace=True)
data.dropna(inplace=True)


data = pd.get_dummies(data, columns=[
    "workclass", "education", "marital_status", "occupation", "relationship",
    "race", "sex", "native_country"
])

data["income"] = data["income"].apply(lambda x: 1 if x == ">50K" else 0)

numerical_columns = ["age", "fnlwgt", "education_num", "capital_gain", "capital_loss", "hours_per_week"]
data[numerical_columns] = (data[numerical_columns] - data[numerical_columns].mean()) / data[numerical_columns].std()

X = data.drop(columns=["income"]).values.astype(np.float32)
y = data["income"].values.astype(np.float32).reshape(-1, 1)

def create_federated_data(X, y, num_clients=10):
    client_data = []
    samples_per_client = len(X) // num_clients
    for i in range(num_clients):
        x_client = X[i * samples_per_client:(i + 1) * samples_per_client]
        y_client = y[i * samples_per_client:(i + 1) * samples_per_client]
        client_data.append((x_client, y_client))
    return client_data

def preprocess(client_data):
    def to_dataset(data):
        x, y = data
        return tf.data.Dataset.from_tensor_slices((x, y)).batch(32)
    return [to_dataset(data) for data in client_data]

federated_data = preprocess(create_federated_data(X, y,5))

def build_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(X.shape[1],)),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(16, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    return model

def model_fn():
    keras_model = build_model()
    input_spec = (
        tf.TensorSpec(shape=[None, X.shape[1]], dtype=tf.float32),
        tf.TensorSpec(shape=[None, 1], dtype=tf.float32)
    )
    return tff.learning.models.from_keras_model(
        keras_model,
        input_spec=input_spec,
        loss=tf.keras.losses.BinaryCrossentropy(),
        metrics=[tf.keras.metrics.BinaryAccuracy()]
    )

client_optimizer_fn = lambda: tf.keras.optimizers.Adam(learning_rate=0.01)
server_optimizer_fn = lambda: tf.keras.optimizers.SGD(learning_rate=0.5)

federated_training_process = tff.learning.algorithms.build_weighted_fed_avg(
    model_fn=model_fn,
    client_optimizer_fn=client_optimizer_fn,
    server_optimizer_fn=server_optimizer_fn
)

state = federated_training_process.initialize()

num_rounds = 15
accuracies, losses = [], []

for round_num in range(1, num_rounds + 1):
    state, metrics = federated_training_process.next(state, federated_data)
    train_metrics = metrics['client_work']['train']
    accuracy = train_metrics.get('binary_accuracy', 0.0)
    loss = train_metrics.get('loss', 0.0)
    accuracies.append(accuracy)
    losses.append(loss)
    print(f"Round {round_num} - Loss: {loss:.4f}, Accuracy: {accuracy:.4f}")

def assign_weights_to_keras_model(keras_model, tff_weights):
    keras_model.set_weights(tff_weights)

final_keras_model = build_model()
global_weights = federated_training_process.get_model_weights(state).trainable
assign_weights_to_keras_model(final_keras_model, global_weights)

final_keras_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
test_data = tf.data.Dataset.from_tensor_slices((X, y)).batch(32)
loss, accuracy = final_keras_model.evaluate(test_data, verbose=0)
print(f"\nFinal Global Model - Loss: {loss:.4f}, Accuracy: {accuracy:.4f}")

Round 1 - Loss: 0.3458, Accuracy: 0.8355
Round 2 - Loss: 0.3299, Accuracy: 0.8463
Round 3 - Loss: 0.3255, Accuracy: 0.8479
Round 4 - Loss: 0.3230, Accuracy: 0.8494
Round 5 - Loss: 0.3205, Accuracy: 0.8505
Round 6 - Loss: 0.3193, Accuracy: 0.8517
Round 7 - Loss: 0.3181, Accuracy: 0.8535
Round 8 - Loss: 0.3169, Accuracy: 0.8538
Round 9 - Loss: 0.3153, Accuracy: 0.8549
Round 10 - Loss: 0.3146, Accuracy: 0.8561
Round 11 - Loss: 0.3134, Accuracy: 0.8558
Round 12 - Loss: 0.3123, Accuracy: 0.8569
Round 13 - Loss: 0.3116, Accuracy: 0.8582
Round 14 - Loss: 0.3108, Accuracy: 0.8586
Round 15 - Loss: 0.3103, Accuracy: 0.8580

Final Global Model - Loss: 0.3030, Accuracy: 0.8606


# ***Vertical Federated Learning***

In [ ]:
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
column_names = [
    "age", "workclass", "fnlwgt", "education", "education_num", "marital_status",
    "occupation", "relationship", "race", "sex", "capital_gain", "capital_loss",
    "hours_per_week", "native_country", "income"
]
data = pd.read_csv(url, header=None, names=column_names, na_values=" ?", skipinitialspace=True)
data.dropna(inplace=True)

data = pd.get_dummies(data, columns=[
    "workclass", "education", "marital_status", "occupation", "relationship",
    "race", "sex", "native_country"
])

data["income"] = data["income"].apply(lambda x: 1 if x == ">50K" else 0)

numerical_columns = ["age", "fnlwgt", "education_num", "capital_gain", "capital_loss", "hours_per_week"]
data[numerical_columns] = (data[numerical_columns] - data[numerical_columns].mean()) / data[numerical_columns].std()

feature_group_a = ["age", "education_num", "capital_gain", "capital_loss"]
feature_group_b = ["fnlwgt", "hours_per_week"] + [col for col in data.columns if col.startswith("workclass")]

X_a = data[feature_group_a].values.astype(np.float32)
X_b = data[feature_group_b].values.astype(np.float32)
y = data["income"].values.astype(np.float32).reshape(-1, 1)

def create_federated_data(X, y):
    return [(X, y)]

def preprocess(client_data):
    def to_dataset(data):
        x, y = data
        return tf.data.Dataset.from_tensor_slices((x, y)).batch(32)
    return [to_dataset(data) for data in client_data]

federated_data = preprocess(create_federated_data(X_a, y))

def build_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(X_a.shape[1],)),
        tf.keras.layers.Dense(16, activation='relu'),
        tf.keras.layers.Dense(8, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    return model

def model_fn():
    keras_model = build_model()
    input_spec = (
        tf.TensorSpec(shape=[None, X_a.shape[1]], dtype=tf.float32),
        tf.TensorSpec(shape=[None, 1], dtype=tf.float32)
    )
    return tff.learning.models.from_keras_model(
        keras_model,
        input_spec=input_spec,
        loss=tf.keras.losses.BinaryCrossentropy(),
        metrics=[tf.keras.metrics.BinaryAccuracy()]
    )

client_optimizer_fn = lambda: tf.keras.optimizers.Adam(learning_rate=0.01)
server_optimizer_fn = lambda: tf.keras.optimizers.SGD(learning_rate=0.5)

federated_training_process = tff.learning.algorithms.build_weighted_fed_avg(
    model_fn=model_fn,
    client_optimizer_fn=client_optimizer_fn,
    server_optimizer_fn=server_optimizer_fn
)

state = federated_training_process.initialize()

num_rounds = 10
for round_num in range(1, num_rounds + 1):
    state, metrics = federated_training_process.next(state, federated_data)
    train_metrics = metrics['client_work']['train']
    accuracy = train_metrics.get('binary_accuracy', 0.0)
    loss = train_metrics.get('loss', 0.0)
    print(f"Round {round_num} - Loss: {loss:.4f}, Accuracy: {accuracy:.4f}")

def assign_weights_to_keras_model(keras_model, tff_weights):
    keras_model.set_weights(tff_weights)

final_keras_model = build_model()
global_weights = federated_training_process.get_model_weights(state).trainable
assign_weights_to_keras_model(final_keras_model, global_weights)

final_keras_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
test_data = tf.data.Dataset.from_tensor_slices((X_a, y)).batch(32)
loss, accuracy = final_keras_model.evaluate(test_data, verbose=0)
print(f"\nFinal Global Model - Loss: {loss:.4f}, Accuracy: {accuracy:.4f}")

Round 1 - Loss: 0.4073, Accuracy: 0.8130
Round 2 - Loss: 0.4005, Accuracy: 0.8171
Round 3 - Loss: 0.3982, Accuracy: 0.8185
Round 4 - Loss: 0.3969, Accuracy: 0.8184
Round 5 - Loss: 0.3961, Accuracy: 0.8193
Round 6 - Loss: 0.3957, Accuracy: 0.8196
Round 7 - Loss: 0.3951, Accuracy: 0.8191
Round 8 - Loss: 0.3950, Accuracy: 0.8192
Round 9 - Loss: 0.3946, Accuracy: 0.8195
Round 10 - Loss: 0.3944, Accuracy: 0.8204

Final Global Model - Loss: 0.3972, Accuracy: 0.8196


# ***TRANSVERSAL FEDERATING LEARNING***

In [ ]:
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
column_names = [
    "age", "workclass", "fnlwgt", "education", "education_num", "marital_status",
    "occupation", "relationship", "race", "sex", "capital_gain", "capital_loss",
    "hours_per_week", "native_country", "income"
]
data = pd.read_csv(url, header=None, names=column_names, na_values=" ?", skipinitialspace=True)
data.dropna(inplace=True)

data = pd.get_dummies(data, columns=[
    "workclass", "education", "marital_status", "occupation", "relationship",
    "race", "sex", "native_country"
])

data["income"] = data["income"].apply(lambda x: 1 if x == ">50K" else 0)

numerical_columns = ["age", "fnlwgt", "education_num", "capital_gain", "capital_loss", "hours_per_week"]
data[numerical_columns] = (data[numerical_columns] - data[numerical_columns].mean()) / data[numerical_columns].std()

num_samples = len(data)
indices = np.random.permutation(num_samples)

client_1_indices = indices[:num_samples // 2]
client_2_indices = indices[num_samples // 2:]

feature_group_a = ["age", "education_num", "capital_gain", "capital_loss"]
feature_group_b = ["fnlwgt", "hours_per_week"] + [col for col in data.columns if col.startswith("workclass")]

X_a_client1 = data.loc[client_1_indices, feature_group_a].values.astype(np.float32)
X_b_client2 = data.loc[client_2_indices, feature_group_b].values.astype(np.float32)
y_client1 = data.loc[client_1_indices, "income"].values.astype(np.float32).reshape(-1, 1)
y_client2 = data.loc[client_2_indices, "income"].values.astype(np.float32).reshape(-1, 1)

def create_federated_data(X, y):
    return [(X, y)]

def preprocess(client_data):
    def to_dataset(data):
        x, y = data
        return tf.data.Dataset.from_tensor_slices((x, y)).batch(32)
    return [to_dataset(data) for data in client_data]

federated_data_client1 = preprocess(create_federated_data(X_a_client1, y_client1))
federated_data_client2 = preprocess(create_federated_data(X_b_client2, y_client2))

def build_model(input_dim):
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(input_dim,)),
        tf.keras.layers.Dense(16, activation='relu'),
        tf.keras.layers.Dense(8, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    return model

def model_fn(input_dim):
    keras_model = build_model(input_dim)
    input_spec = (
        tf.TensorSpec(shape=[None, input_dim], dtype=tf.float32),
        tf.TensorSpec(shape=[None, 1], dtype=tf.float32)
    )
    return tff.learning.models.from_keras_model(
        keras_model,
        input_spec=input_spec,
        loss=tf.keras.losses.BinaryCrossentropy(),
        metrics=[tf.keras.metrics.BinaryAccuracy()]
    )

client_optimizer_fn = lambda: tf.keras.optimizers.Adam(learning_rate=0.01)
server_optimizer_fn = lambda: tf.keras.optimizers.SGD(learning_rate=0.5)

federated_training_process_client1 = tff.learning.algorithms.build_weighted_fed_avg(
    model_fn=lambda: model_fn(len(feature_group_a)),
    client_optimizer_fn=client_optimizer_fn,
    server_optimizer_fn=server_optimizer_fn
)

federated_training_process_client2 = tff.learning.algorithms.build_weighted_fed_avg(
    model_fn=lambda: model_fn(len(feature_group_b)),
    client_optimizer_fn=client_optimizer_fn,
    server_optimizer_fn=server_optimizer_fn
)

state_client1 = federated_training_process_client1.initialize()
state_client2 = federated_training_process_client2.initialize()

num_rounds = 10
for round_num in range(1, num_rounds + 1):
    state_client1, metrics1 = federated_training_process_client1.next(state_client1, federated_data_client1)
    state_client2, metrics2 = federated_training_process_client2.next(state_client2, federated_data_client2)

    loss1, acc1 = metrics1['client_work']['train'].get('loss', 0.0), metrics1['client_work']['train'].get('binary_accuracy', 0.0)
    loss2, acc2 = metrics2['client_work']['train'].get('loss', 0.0), metrics2['client_work']['train'].get('binary_accuracy', 0.0)

    print(f"Round {round_num}: Client 1 - Loss: {loss1:.4f}, Accuracy: {acc1:.4f} | Client 2 - Loss: {loss2:.4f}, Accuracy: {acc2:.4f}")

def assign_weights_to_keras_model(keras_model, tff_weights):
    keras_model.set_weights(tff_weights)

final_model_client1 = build_model(len(feature_group_a))
final_model_client2 = build_model(len(feature_group_b))

global_weights_client1 = federated_training_process_client1.get_model_weights(state_client1).trainable
global_weights_client2 = federated_training_process_client2.get_model_weights(state_client2).trainable

assign_weights_to_keras_model(final_model_client1, global_weights_client1)
assign_weights_to_keras_model(final_model_client2, global_weights_client2)

final_model_client1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
final_model_client2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

test_data_client1 = tf.data.Dataset.from_tensor_slices((X_a_client1, y_client1)).batch(32)
test_data_client2 = tf.data.Dataset.from_tensor_slices((X_b_client2, y_client2)).batch(32)

loss1, acc1 = final_model_client1.evaluate(test_data_client1, verbose=0)
loss2, acc2 = final_model_client2.evaluate(test_data_client2, verbose=0)

print(f"\nFinal Models - Client 1: Loss {loss1:.4f}, Accuracy {acc1:.4f} | Client 2: Loss {loss2:.4f}, Accuracy {acc2:.4f}")

Round 1: Client 1 - Loss: 0.4159, Accuracy: 0.8101 | Client 2 - Loss: 0.5155, Accuracy: 0.7569
Round 2: Client 1 - Loss: 0.4068, Accuracy: 0.8116 | Client 2 - Loss: 0.5106, Accuracy: 0.7581
Round 3: Client 1 - Loss: 0.4032, Accuracy: 0.8140 | Client 2 - Loss: 0.5088, Accuracy: 0.7597
Round 4: Client 1 - Loss: 0.4013, Accuracy: 0.8141 | Client 2 - Loss: 0.5082, Accuracy: 0.7603
Round 5: Client 1 - Loss: 0.4004, Accuracy: 0.8147 | Client 2 - Loss: 0.5074, Accuracy: 0.7603
Round 6: Client 1 - Loss: 0.3997, Accuracy: 0.8144 | Client 2 - Loss: 0.5068, Accuracy: 0.7608
Round 7: Client 1 - Loss: 0.3994, Accuracy: 0.8152 | Client 2 - Loss: 0.5064, Accuracy: 0.7614
Round 8: Client 1 - Loss: 0.3987, Accuracy: 0.8158 | Client 2 - Loss: 0.5059, Accuracy: 0.7616
Round 9: Client 1 - Loss: 0.3985, Accuracy: 0.8160 | Client 2 - Loss: 0.5058, Accuracy: 0.7622
Round 10: Client 1 - Loss: 0.3985, Accuracy: 0.8172 | Client 2 - Loss: 0.5057, Accuracy: 0.7617

Final Models - Client 1: Loss 0.4041, Accuracy 0

# ***PART 02: CONTRACT USING SOLIDITY***